# 교통사고 데이터

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data/Seoul_Traffic_Accident_20172019.csv/Seoul_Traffic_Accident_20172019.csv')
data

,Unnamed: 0,발생일,발생시간,발생지_시도,발생지_시군구,법정동명,사고건수,사망자수,중상자수,경상자수,부상신고자수
0,0,2017-01-01,0,서울,강북구,미아동,1,0,1,6,0
1,1,2017-01-01,0,서울,노원구,상계동,1,0,0,1,0
2,2,2017-01-01,0,서울,동작구,상도동,1,0,0,1,0
3,3,2017-01-01,0,서울,서초구,서초동,1,0,0,1,0
4,4,2017-01-01,0,서울,송파구,잠실동,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
114438,114438,2019-12-31,22,서울,송파구,송파동,1,0,0,1,0
114439,114439,2019-12-31,22,서울,영등포구,대림동,1,0,0,1,0
114440,114440,2019-12-31,22,서울,중랑구,면목동,1,0,0,0,1
114441,114441,2019-12-31,23,서울,강남구,수서동,1,0,0,1,0


In [3]:
data = data.drop(["Unnamed: 0"], axis=1)

data.loc[:, "year"] = data["발생일"].str[:4].astype(int)
data.loc[:, "month"] = data["발생일"].str[5:7].astype(int)
data.loc[:, "date"] = data["발생일"].str[8:].astype(int)

front = ["year", "month", "date"]
others = [c for c in data.columns if c not in front]
data = data[front + others]

data = data.drop(["발생일"], axis=1)

data.info()

<class 'pandas.DataFrame'>
RangeIndex: 114443 entries, 0 to 114442
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   year     114443 non-null  int64
 1   month    114443 non-null  int64
 2   date     114443 non-null  int64
 3   발생시간     114443 non-null  int64
 4   발생지_시도   114443 non-null  str  
 5   발생지_시군구  114443 non-null  str  
 6   법정동명     114438 non-null  str  
 7   사고건수     114443 non-null  int64
 8   사망자수     114443 non-null  int64
 9   중상자수     114443 non-null  int64
 10  경상자수     114443 non-null  int64
 11  부상신고자수   114443 non-null  int64
dtypes: int64(9), str(3)
memory usage: 10.5 MB


In [4]:
data = data.drop(['발생시간', '발생지_시도', '부상신고자수'], axis=1)

In [5]:
agg_dict = {
    "사고건수":"sum",
    "사망자수":"sum",
    "중상자수":"sum",
    "경상자수":"sum"
}
datas_by_month = data.groupby(["year", "month", "date"]).agg(agg_dict)
datas_by_month

사고건수  사망자수  중상자수  경상자수
year month date                        
2017 1     1       73     2    25    90
           2       82     4    27    92
           3       88     1    25    80
           4      115     1    39    99
           5      103     1    34   111
...               ...   ...   ...   ...
2019 12    27      94     0    27    88
           28      94     0    27   105
           29      78     1    26    87
           30      91     1    19    93
           31     106     2    20   102

[1095 rows x 4 columns]

In [7]:
agg_dict = {
    "사고건수":"sum",
    "사망자수":"sum",
    "중상자수":"sum",
    "경상자수":"sum"
}
datas_by_zone = data.groupby(["발생지_시군구", "법정동명"]).agg(agg_dict)
datas_by_zone.loc[:, "per_death"] = (datas_by_zone["사망자수"] / datas_by_zone["사고건수"] * 100).round(2)
datas_by_zone.loc[:, "per_fatal"] = (datas_by_zone["중상자수"] / datas_by_zone["사고건수"] * 100).round(2)
# datas_by_zone = datas_by_zone.reset_index()
datas_by_zone[datas_by_zone["사고건수"] >= 100]

사고건수  사망자수  중상자수  경상자수  per_death  per_fatal
발생지_시군구 법정동명                                              
강남구     개포동    564     5   165   566       0.89      29.26
        논현동   1801     4   384  2004       0.22      21.32
        대치동   1334     4   380  1233       0.30      28.49
        도곡동    641     3   186   632       0.47      29.02
        삼성동    963     4   253  1029       0.42      26.27
...            ...   ...   ...   ...        ...        ...
중랑구     면목동   1707    10   487  1531       0.59      28.53
        묵동     508     2   190   383       0.39      37.40
        상봉동    699     6   226   624       0.86      32.33
        신내동    568     4   174   549       0.70      30.63
        중화동    609     2   193   546       0.33      31.69

[211 rows x 6 columns]